In [1]:
import numpy as np
import math

from qiskit import QuantumCircuit
import qiskit.quantum_info as qi


from hamming import *
from loss import *

In [2]:
N_xi, N_features = 6, 8
xi = np.random.randint(2, size=(N_xi, N_features))
xi[ xi == 0 ] = -1   # data is encoded as +- 1

labels = np.ones((N_features, ))

print(xi.shape)

(6, 8)


In [3]:
HH = Hamming(n_qubit = 8, n_ancillary = 4, dataset = xi)

qc = QuantumCircuit( HH.n_qubit + HH.n_ancillary )

# init the circuit state
for ii in range(HH.n_qubit):  qc.h(ii)

qc.x(8+3)

#qc.barrier()

In [4]:
qc.draw()

┌───┐
 q_0: ┤ H ├
      ├───┤
 q_1: ┤ H ├
      ├───┤
 q_2: ┤ H ├
      ├───┤
 q_3: ┤ H ├
      ├───┤
 q_4: ┤ H ├
      ├───┤
 q_5: ┤ H ├
      ├───┤
 q_6: ┤ H ├
      ├───┤
 q_7: ┤ H ├
      └───┘
 q_8: ─────
           
 q_9: ─────
           
q_10: ─────
      ┌───┐
q_11: ┤ X ├
      └───┘

In [5]:
P = 100
dt = 1

for pp in range(2):
    s_p = (pp+1)/P
    gamma_p = s_p*dt
    beta_p = (1-s_p)*dt

    for mu in range( HH.data_size ):
        #HH.hamming_circuit_for_sample(mu, qc = qc)
        HH.control_unitaries(mu, gamma = gamma_p, qc = qc)

    # apply Ux
    for ii in range( HH.n_qubit ):
    #    # syntax:  lambda, control_qubit, target_qubit
        qc.u( 2 * beta_p, math.pi/2, -math.pi/2, ii)

    # measure loss
    get_statevec(qc, HH.n_qubit)


simulator = Aer.get_backend('aer_simulator_statevector')
circ = transpile(qc, simulator)

# Run and get counts
result = simulator.run(circ).result()
res = get_losses_from_results(result, xi, labels, HH.n_qubit, representation='burla')


Constructing H_perc: 100%|██████████| 6/6 [00:00<00:00, 240.02it/s]


In [6]:
res

array([0.+0.j, 0.+0.j])

In [7]:
qc.draw()

┌───┐┌───────────────┐                                   »
 q_0: ┤ H ├┤ Rz(0.0070711) ├───────────────────────────────────»
      ├───┤└───────┬───────┘┌────────────────┐                 »
 q_1: ┤ H ├────────┼────────┤ Rz(-0.0070711) ├─────────────────»
      ├───┤        │        └───────┬────────┘┌───────────────┐»
 q_2: ┤ H ├────────┼────────────────┼─────────┤ Rz(0.0070711) ├»
      ├───┤        │                │         └───────┬───────┘»
 q_3: ┤ H ├────────┼────────────────┼─────────────────┼────────»
      ├───┤        │                │                 │        »
 q_4: ┤ H ├────────┼────────────────┼─────────────────┼────────»
      ├───┤        │                │                 │        »
 q_5: ┤ H ├────────┼────────────────┼─────────────────┼────────»
      ├───┤        │                │                 │        »
 q_6: ┤ H ├────────┼────────────────┼─────────────────┼────────»
      ├───┤        │                │                 │        »
 q_7: ┤ H ├────────┼────────────────┼─────────────────┼────────»
      └───┘        │                │                 │        »
 q_8: ─────────────┼────────────────┼─────────────────┼────────»
                   │                │                 │        »
 q_9: ─────────────┼────────────────┼─────────────────┼────────»
                   │                │                 │        »
q_10: ─────────────┼────────────────┼─────────────────┼────────»
      ┌───┐        │                │                 │        »
q_11: ┤ X ├────────■────────────────■─────────────────■────────»
      └───┘                                                    »
«                                                                           »
« q_0: ─────────────────────────────────────────────────────────────────────»
«                                                                           »
« q_1: ─────────────────────────────────────────────────────────────────────»
«                                                                           »
« q_2: ─────────────────────────────────────────────────────────────────────»
«      ┌───────────────┐                                                    »
« q_3: ┤ Rz(0.0070711) ├────────────────────────────────────────────────────»
«      └───────┬───────┘┌───────────────┐                                   »
« q_4: ────────┼────────┤ Rz(0.0070711) ├───────────────────────────────────»
«              │        └───────┬───────┘┌────────────────┐                 »
« q_5: ────────┼────────────────┼────────┤ Rz(-0.0070711) ├─────────────────»
«              │                │        └───────┬────────┘┌───────────────┐»
« q_6: ────────┼────────────────┼────────────────┼─────────┤ Rz(0.0070711) ├»
«              │                │                │         └───────┬───────┘»
« q_7: ────────┼────────────────┼────────────────┼─────────────────┼────────»
«              │                │                │                 │        »
« q_8: ────────┼────────────────┼────────────────┼─────────────────┼────────»
«              │                │                │                 │        »
« q_9: ────────┼────────────────┼────────────────┼─────────────────┼────────»
«              │                │                │                 │        »
«q_10: ────────┼────────────────┼────────────────┼─────────────────┼────────»
«              │                │                │                 │        »
«q_11: ────────■────────────────■────────────────■─────────────────■────────»
«                                                                           »
«                        ┌────────────────┐                                  »
« q_0: ──────────────────┤ Rz(-0.0070711) ├──────────────────────────────────»
«                        └───────┬────────┘┌───────────────┐                 »
« q_1: ──────────────────────────┼─────────┤ Rz(0.0070711) ├─────────────────»
«                                │         └───────┬───────┘┌───────────────┐»
« q_2: ──────────────────────────┼──

In [8]:
rho = qi.Statevector.from_instruction(qc)

rhosys = qi.partial_trace(rho,[ HH.n_qubit + ii for ii in range(HH.n_ancillary)] )

QiskitError: 'Cannot apply Instruction: snapshot'

In [ ]:
rhosys

DensityMatrix([[ 3.55201644e-03+9.03923797e-22j,
                -6.87560957e-19-4.36702105e-04j,
                -2.17894614e-18+2.74145404e-03j, ...,
                -4.51404150e-19+2.41026274e-04j,
                -4.15046144e-20-1.93398868e-04j,
                 9.46796988e-04+5.59041745e-19j],
               [-6.87560957e-19+4.36702105e-04j,
                 1.53810159e-03+4.07090275e-22j,
                 2.55248061e-03-2.17781930e-19j, ...,
                 1.05661732e-03+3.03804271e-19j,
                 1.11454528e-03+4.92973175e-19j,
                -3.10014059e-19+1.93398868e-04j],
               [-2.17894614e-18-2.74145404e-03j,
                 2.55248061e-03+2.18222113e-19j,
                 1.61946278e-02+1.91046108e-21j, ...,
                 7.37738137e-04+4.02364531e-19j,
                 1.05661732e-03+3.43048344e-19j,
                 2.71050543e-20-2.41026274e-04j],
               ...,
               [-4.51404150e-19-2.41026274e-04j,
                 1.05661732e-03